# 1) Uniendo todos los registros con manifiestos

In [ ]:
import os

import pandas as pd

import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

from math import isnan

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#para navegar dentro de las carpetas de drive:
#cd drive/ #cambia el directorio actual
#!ls #lista los archivos del directorio actual

In [ ]:
mt_path = "drive/MyDrive/"
path = sorted([os.path.join(mt_path, file)
                          for file in os.listdir(mt_path)
                          if file.endswith('.csv') and file.startswith('residuos_especiales')])

In [ ]:
#path_csv_rrpp = str('{}residuos_especiales_part1.csv'.format(mt_path))
path_csv_rrpp = str('{}residuos_especiales_part2.csv'.format(mt_path))

In [ ]:
len(path)

170

In [ ]:
dfrrpp_intermedio = pd.DataFrame()

# se busca el csv original
for i in path:
  #print(path[i])
  dfrrpp = pd.read_csv(i, sep=",")
  # se agrega la fila correspondiente al manifiesto que se está revisando en este ciclo
  dfrrpp_intermedio = dfrrpp_intermedio.append(dfrrpp, ignore_index=True)

In [ ]:
dfrrpp_intermedio.shape

In [ ]:
dfrrpp_intermedio.to_csv(path_csv_rrpp, index=False, encoding='utf8')

# 2) Juntando los archivos completos de cada drive
Una vez corrida la primera parte del código en cada uno de los drives con registros de manifiestos, se carga cada csv completo en el mismo drive y se apendean en un sólo csv.

In [ ]:
path2 = sorted([os.path.join(mt_path, file)
                          for file in os.listdir(mt_path)
                          if file.endswith('.csv') and file.startswith('residuos_especiales_part')])

In [ ]:
path2[0:3]

In [ ]:
path_csv_rrpp_comp = str('{}residuos_especiales_completo.csv'.format(mt_path))

In [ ]:
dfrrpp_completo = pd.DataFrame()

# se busca el csv original
for i in path2:
  #print(path[i])
  dfrrpp = pd.read_csv(i, sep=",")
  # se agrega la fila correspondiente al manifiesto que se está revisando en este ciclo
  dfrrpp_completo = dfrrpp_completo.append(dfrrpp, ignore_index=True)

In [ ]:
dfrrpp_completo.shape #se revisa el tamaño del df

In [ ]:
dfrrpp_completo.to_csv(path_csv_rrpp_comp, index=False, encoding='utf8') #se guarda el df

# 3) Limpiando el dataframe completo

Si ya se corrieron las secciones anteriores se puede comenzar a trabajar desde aquí, cargardo el df completo

In [ ]:
dfrrpp_completo = pd.read_csv(path_csv_rrpp_comp, sep=",") #se carga el df completo

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (48,60,67,73,80,87,93,94,95,96,98,99,106,113,114,115,116,118,119,120,121,122,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,151,152,153,154,155,157,158,159,160,161,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,218,219,220,221,222,224,225,226,227,228,230,231,232,233,234,236,237,238,239,240,242,243,244,245,246,248,249,250,251,252,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,309,310,311,312,313,315,316,317,318,319,321,322,323,324,325,327,328,329,330,331,333,334,335,336,337,339,340,341,342,343,345,346

In [ ]:
dfrrpp_completo.head()

In [ ]:
dfrrpp_completo.shape

In [ ]:
dfrrpp_completo.columns

In [ ]:
dfrrpp_completo.drop_duplicates(inplace=True)

In [ ]:
dfrrpp_completo.estado.unique()

In [ ]:
#se revisan los tamaños del data según distintos filtros
print(dfrrpp_completo.shape)
print(dfrrpp_completo[dfrrpp_completo['estado']=='CERRADO'].shape)

In [ ]:
dfrrpp_completo = dfrrpp_completo[dfrrpp_completo['estado']=='CERRADO']

In [ ]:
cols_cantidad = [col for col in dfrrpp_completo.columns if 'cantidad_verif' in col]

In [ ]:
cols_cantidad

In [ ]:
dfrrpp_completo[cols_cantidad].head()

In [ ]:
dfrrpp_completo[cols_cantidad].sum(axis=1) #se suman las columnas de cantidades

In [ ]:
dfrrpp_completo['cantidad_total'] = dfrrpp_completo[cols_cantidad].sum(axis=1)

In [ ]:
cols_utiles = ['numero', 'fecha_retiro', 'cantidad_total'] #se arma una lista con los nombres de la cols a seleccionar

In [ ]:
cols_utiles

In [ ]:
dfrrpp_resumido = dfrrpp_completo[cols_utiles] #se filtran las columnas que se van a utilizar para el modelo

In [ ]:
dfrrpp_resumido.shape #se revisa la forma del df

In [ ]:
dfrrpp_resumido.head()

In [ ]:
dfrrpp_resumido['fecha_retiro'] = dfrrpp_resumido['fecha_retiro'].map(lambda x: x.lstrip('Fecha de Viaje: ').rstrip())

In [ ]:
!pip install dateparser #se instala el paquete para trabajar con str con formato de fechas
import dateparser #se importa el paquete

In [ ]:
dfrrpp_resumido['fecha_retiro_2'] = dfrrpp_resumido.fecha_retiro.map(dateparser.parse) #se mapea cada una de las fechas y se modifica a un formato más estandarizado

In [ ]:
dfrrpp_resumido.head() #se revisan los cambios

In [ ]:
dfrrpp_resumido.drop(labels='fecha_retiro', axis=1, inplace=True) #se dropea la columna anterior de fecha

In [ ]:
print(pd.DatetimeIndex(dfrrpp_resumido['fecha_retiro_2']).year)

In [ ]:
dfrrpp_resumido.sort_values('fecha_retiro_2', ascending=False) #se buscan años atípicos

In [ ]:
dfrrpp_resumido['year'] = pd.DatetimeIndex(dfrrpp_resumido['fecha_retiro_2']).year #se agrega la columna con el año

In [ ]:
dfrrpp_resumido.groupby('year').count() #se revisa la cantidad de manifiestos por año

In [ ]:
#dfrrpp_resumido[(dfrrpp_resumido['year']<2022) & (dfrrpp_resumido['year']>2015)].shape 
dfrrpp_resumido = dfrrpp_resumido[(dfrrpp_resumido['year']<2022) & (dfrrpp_resumido['year']>2015)] #se seleccionan los años con una distribución más homogénea: 2016 a 2021

In [ ]:
dfrrpp_resumido.head()

In [ ]:
dfrrpp_resumido.rename(columns={'fecha_retiro_2':'fecha_retiro'}, inplace=True) #por ultimo se cambia el nombre de la columna de fecha

# 4) Guardado del dataset resumido

In [ ]:
path_csv_rrpp_resum = str('{}residuos_especiales_resumido.csv'.format(mt_path))

In [ ]:
dfrrpp_resumido.to_csv(path_csv_rrpp_resum, index=False, encoding='utf8') #se guarda el df

# 5) Filtrando para descargar y trabajar localmente en R

In [ ]:
dfrrpp_completo = pd.read_csv(path_csv_rrpp_comp, sep=",") #se carga el df completo

NameError: ignored

In [ ]:
dfrrpp_completo.head()

,numero,estado,certificado,fecha_retiro,tipo_mt,origen,grei,cuit_generador,razon_social_generador,domicilio_generador,...,caracteristica_verif_24,cantidad_verif_24,estado_fisico_verif_24,observaciones_verif_24,composicion_verif_25,categoria_verif_25,caracteristica_verif_25,cantidad_verif_25,estado_fisico_verif_25,observaciones_verif_25
0,3000312,CERRADO,875263,Fecha de Viaje: 30/05/2016,Especiales,Generador,5810.0,30-70759184/2 (Id:11081),GAS LANUS S.A.,25 DE MAYO Nº : 883 Ruta: : 210 (PAV?N),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3000313,CERRADO,902328,Fecha de Viaje: 31/05/2016,Especiales,Generador,NaN,30-67813830/0 (Id:55627),WAL MART ARGENTINA S.R.L.,SAAVEDRA Nº : S/N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3000316,CERRADO,856441,Fecha de Viaje: 24/05/2016,Especiales,Generador,NaN,30-50336489/8 (Id:53866),SIEMENS SA,RUTA 3 Nº : KM 704 Ruta: : RUTA 3 Km:: 704,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3000317,VALIDADO,pendiente,Fecha de Viaje: 27/05/2016,Especiales,Generador,NaN,30-71161663/9 (Id:41589),CONCENTRADOS Y COMPUESTOS S.A.,23 Nº : 431 Ruta: : 8 Km:: 60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3000321,CERRADO,902426,Fecha de Viaje: 27/05/2016,Especiales,Generador,NaN,30-70969422/3 (Id:),G P PHARM S.A.,PANAMA Nº : 2121,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dfrrpp_completo.shape

(544720, 347)

In [ ]:
dfrrpp_completo = dfrrpp_completo[dfrrpp_completo['estado']=='CERRADO'] #se filtra por los manifiestos "cerrados"

In [ ]:
dfrrpp_completo.shape

(418787, 347)

In [ ]:
dfrrpp_completo['fecha_retiro'] = dfrrpp_completo['fecha_retiro'].map(lambda x: x.lstrip('Fecha de Viaje: ').rstrip())

In [ ]:
dfrrpp_completo['fecha_retiro'].str.contains(pat = "2021|2020|2019|2018|2017|2016").sum() #se calcula la cantidad de registros para el período 2016-2021

471742

In [ ]:
dfrrpp_completo = dfrrpp_completo[(dfrrpp_completo['fecha_retiro'].str.contains(pat = "2021|2020|2019|2018|2017|2016"))] #se filtra el df

In [ ]:
dfrrpp_completo['fecha_retiro'].str.contains(pat = "2022|2015").sum() #se revisa que no queden registros de otros años

0

In [ ]:
dfrrpp_completo.info(memory_usage='deep') #estimación del peso del df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395325 entries, 0 to 514950
Columns: 347 entries, numero to observaciones_verif_25
dtypes: float64(28), int64(1), object(318)
memory usage: 4.4 GB


In [ ]:
#hace falta achicar más el df. Se prueba eliminando algunas columnas
dfrrpp_completo.drop('estado', inplace=True, axis=1)
dfrrpp_completo.drop('tipo_mt', inplace=True, axis=1)
dfrrpp_completo.drop('origen', inplace=True, axis=1)
dfrrpp_completo.drop('grei', inplace=True, axis=1)

In [ ]:
dfrrpp_completo.columns.shape #cantidad de columnas: demasiadas

(343,)

In [ ]:
dfrrpp_completo.filter(regex=("numero|certificado|fecha_retiro|generador$|transportista$|destino$|_verif_1$|_verif_2$|_verif_3$|_verif_4$|_verif_5$|_verif_6$|_verif_7$|_verif_8$|_verif_9$|_verif_10$")).info(memory_usage='deep') #estimación del peso del df
#se observa, como era de esperar, que a medida que aumenta el orden del residuo, mayor cantidad de nulos se encuentran
#por esto se opta por eliminar los ordenes mayores a 3, tomando una representación de al menos 5% del total de registros

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395325 entries, 0 to 514950
Data columns (total 78 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   numero                      395325 non-null  int64  
 1   certificado                 395325 non-null  object 
 2   fecha_retiro                395325 non-null  object 
 3   cuit_generador              395325 non-null  object 
 4   razon_social_generador      395325 non-null  object 
 5   domicilio_generador         395325 non-null  object 
 6   localidad_generador         395325 non-null  object 
 7   cuit_transportista          395325 non-null  object 
 8   che_transportista           395325 non-null  float64
 9   razon_social_transportista  395325 non-null  object 
 10  domicilio_transportista     395325 non-null  object 
 11  localidad_transportista     395325 non-null  object 
 12  destino                     395325 non-null  object 
 13  che_destino   

In [ ]:
dfrrpp_res = dfrrpp_completo.filter(regex=("numero|certificado|fecha_retiro|generador$|transportista$|destino$|_verif_1$|_verif_2$|_verif_3$")) #se filtra el df completo y se guarda en un nuevo df


In [ ]:
dfrrpp_res.info(memory_usage='deep') #estimación del peso del df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395325 entries, 0 to 514950
Data columns (total 36 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   numero                      395325 non-null  int64  
 1   certificado                 395325 non-null  object 
 2   fecha_retiro                395325 non-null  object 
 3   cuit_generador              395325 non-null  object 
 4   razon_social_generador      395325 non-null  object 
 5   domicilio_generador         395325 non-null  object 
 6   localidad_generador         395325 non-null  object 
 7   cuit_transportista          395325 non-null  object 
 8   che_transportista           395325 non-null  float64
 9   razon_social_transportista  395325 non-null  object 
 10  domicilio_transportista     395325 non-null  object 
 11  localidad_transportista     395325 non-null  object 
 12  destino                     395325 non-null  object 
 13  che_destino   

In [ ]:
!pip install dateparser #se instala el paquete para trabajar con str con formato de fechas
import dateparser #se importa el paquete

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 292 kB 15.6 MB/s 


In [ ]:
dfrrpp_res['fecha_retiro_2'] = dfrrpp_res.fecha_retiro.map(dateparser.parse) #se mapea cada una de las fechas y se modifica a un formato más estandarizado

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
dfrrpp_res.drop(labels='fecha_retiro', axis=1, inplace=True) #se dropea la columna anterior de fecha

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
dfrrpp_res.rename(columns={'fecha_retiro_2':'fecha_retiro'}, inplace=True) #por ultimo se cambia el nombre de la columna de fecha

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
dfrrpp_res.head()

,numero,certificado,cuit_generador,razon_social_generador,domicilio_generador,localidad_generador,cuit_transportista,che_transportista,razon_social_transportista,domicilio_transportista,...,cantidad_verif_2,estado_fisico_verif_2,observaciones_verif_2,composicion_verif_3,categoria_verif_3,caracteristica_verif_3,cantidad_verif_3,estado_fisico_verif_3,observaciones_verif_3,fecha_retiro
0,3000312,875263,30-70759184/2 (Id:11081),GAS LANUS S.A.,25 DE MAYO Nº : 883 Ruta: : 210 (PAV?N),LANUS,30-71008458/7 (Id:37494),355.0,LA RUMANA S.R.L.,LOS PINOS Nº : 155,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-05-30
1,3000313,902328,30-67813830/0 (Id:55627),WAL MART ARGENTINA S.R.L.,SAAVEDRA Nº : S/N,MORENO,30-70845639/6 (Id:37363),247.0,FRANCO TRANSPORTES S.R.L,FLORENCIO SANCHEZ Nº : 450,...,450.0,Solido,NaN,Residuos especiales,Y31 - Desechos que tengan como Constituyentes:...,H13 -,100.0,Solido,NaN,2016-05-31
2,3000316,856441,30-50336489/8 (Id:53866),SIEMENS SA,RUTA 3 Nº : KM 704 Ruta: : RUTA 3 Km:: 704,BAHIA BLANCA,20-13549430/6 (Id:37593),356.0,"GARNICA, JULIO CESAR",LEONCIO MANSILLA Nº : 1787,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-05-24
4,3000321,902426,30-70969422/3 (Id:),G P PHARM S.A.,PANAMA Nº : 2121,MARTINEZ,30-71439504/8 (Id:53784),486.0,MD SERVICIOS SRL,SAN JUAN Nº : 2090,...,28.0,Liquido,Almacenado en cuba.,NaN,NaN,NaN,NaN,NaN,NaN,2016-05-27
7,3000325,880034,30-50104769/0 (Id:24747),SEALED AIR ARGENTINA S.A.,( 26 ) PRIMERA JUNTA Nº : 550,QUILMES,30-71123997/5 (Id:37451),51.0,HOUSE AND TRUCK S.A.,MARAÑÓN Nº : 840,...,640.0,Solido,NaN,Agua con hidrocarburos,Y9 - Mezclas y emulsiones de desecho de aceite...,H12 -,400.0,Liquido,NaN,2016-05-24


In [ ]:
path_csv_rrpp_res = str('{}manifiestos_rree.csv'.format(mt_path))

In [ ]:
dfrrpp_res.to_csv(path_csv_rrpp_res, index=False, encoding='utf8') #se guarda el df